In [ ]:
# !pip install pandas
# !pip install hana_ml
# !pip install generative-ai-hub-sdk

In this notebook, we will run a basic Q&A using Retrieval Augmented Generation.<br>
The context is retrieved via a vector search in HANA Cloud.<br>
First, we import some data from a csv file (which contains vector embeddings generated by text-embedding-ada-002).<br>
Then we define a run_vector_search function which is is used in the ask_llm function to provide context which is infused into the prompt.<br>

You need to set up SAP AI Core and generate "deployments" for the services you want to use. See https://github.wdf.sap.corp/AI/generative-ai-hub-sdk/tree/main/docs/proxy.

Import some vector data from csv file

In [ ]:
# import some vector data from csv
import pandas as pd
df = pd.read_csv('./data/GRAPH_DOCU_QRC3.csv', low_memory=False)
df.head(3)

In [ ]:
# connect to HANA
from hana_ml.dataframe import ConnectionContext
# cc = ConnectionContext(userkey='GR3', encrypt=True)
cc= ConnectionContext(
    address='[somehost].hanacloud.ondemand.com', 
    port='443', 
    user='[your user]', 
    password='[your password]', 
    encrypt=True
    )
print(cc.hana_version())
print(cc.get_current_schema())

In [4]:
# # Drop table if already exists
# cursor = cc.connection.cursor()
# sql_command = '''DROP TABLE GRAPH_DOCU_QRC3;'''
# cursor.execute(sql_command)
# cursor.close()

In [5]:
# Create a table
cursor = cc.connection.cursor()
sql_command = '''CREATE TABLE GRAPH_DOCU_QRC3(ID BIGINT, L1 NVARCHAR(3), L2 NVARCHAR(3), L3 NVARCHAR(3), FILENAME NVARCHAR(100), HEADER1 NVARCHAR(5000), HEADER2 NVARCHAR(5000), TEXT NCLOB, VECTOR_STR NCLOB);'''
cursor.execute(sql_command)
cursor.close()

In [ ]:
# Upload data
from hana_ml.dataframe import create_dataframe_from_pandas
v_hdf = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=df,
    table_name="GRAPH_DOCU_QRC3", 
    allow_bigint=True,
    append=True
    )

In [7]:
# Add REAL_VECTOR column
cursor = cc.connection.cursor()
sql_command = '''ALTER TABLE GRAPH_DOCU_QRC3 ADD (VECTOR REAL_VECTOR(1536));'''
cursor.execute(sql_command)
cursor.close()

In [8]:
# Create vectors from strings
cursor = cc.connection.cursor()
sql_command = '''UPDATE GRAPH_DOCU_QRC3 SET VECTOR = TO_REAL_VECTOR(VECTOR_STR);'''
cursor.execute(sql_command)
cursor.close()

RAG using gen ai hub sdk and HANA Vector Search

In [9]:
# Get embeddings
from gen_ai_hub.proxy.native.openai import embeddings

def get_embedding(input, model_name="text-embedding-ada-002") -> str:
    response = embeddings.create(
        input=input,
        model_name=model_name, 
        )
    return response.data[0].embedding

In [10]:
# Wrapping HANA vector search in a function
def run_vector_search(query: str, metric="COSINE_SIMILARITY", k=4):
    if metric == 'L2DISTANCE':
        sort = 'ASC'
    else:
        sort = 'DESC'
    query_vector = get_embedding(query)
    sql = '''SELECT TOP {k} "ID", "HEADER1", "HEADER2", "TEXT"
        FROM "GRAPH_DOCU_QRC3"
        ORDER BY "{metric}"("VECTOR", TO_REAL_VECTOR('{qv}')) {sort}'''.format(k=k, metric=metric, qv=query_vector, sort=sort)
    hdf = cc.sql(sql)
    df_context = hdf.head(k).collect()
    # context = ' '.join(df_context['TEXT'].astype('string'))
    return df_context

In [ ]:
# Test the vector search
query = "How can I run a shortest path algorithm?"
df_context = run_vector_search(query=query, metric="COSINE_SIMILARITY",k=4)
df_context

In [18]:
from gen_ai_hub.proxy.native.openai import chat

# basic LLM prompt for RAG
sys_content = '''Your task is to answer the question using the provided context wrapped in triple quotes. 
If the provided context does not contain the information needed to answer this question then come up with your own answer. '''

def ask_llm(query: str, retrieval_augmented_generation: bool, metric='COSINE_SIMILARITY', k = 4) -> str:
    context = ''
    if retrieval_augmented_generation == True:
        df_context = run_vector_search(query, metric, k)
        context = ' '.join(df_context['TEXT'].astype('string'))
    user_content = '"""' + context + '"""'+ ' Question: ' + query
    messages=[{"role": "system", "content": sys_content}, {"role": "user", "content": user_content}]
    
    # kwargs = dict(model_name='gpt-4', messages=messages)
    kwargs = dict(model_name='gpt-35-turbo', messages=messages)
    
    response = chat.completions.create(**kwargs)
    
    return response

In [ ]:
# query = "Can you define a HANA graph workspace on a JSON document store collection?"
#query = "How can I define a HANA graph workspace on a JSON document store collection?"
#query = "How do you run a shortest path algorithm in SAP HANA Graph engine?"
# query = "How can I run community detection Louvain in SAP HANA Graph?"
# query = "How can I run a BFS traversal in HANA Cloud"
query = "I want to calculate a shortest path. How do I do that?"

response = ask_llm(query=query, retrieval_augmented_generation=True, k=4)

print(response.choices[0].message.content)